<a href="https://colab.research.google.com/github/rryl/amazon_vine_analysis/blob/main/vine_review_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [90.7 kB]
Get:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:13 https://dev

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-08-15 23:10:49--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.23MB/s    in 0.2s    

2022-08-15 23:10:49 (5.23 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_01.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22480053|R28HBXXO1UEVJT|0843952016|      34858117|          The Rising|           Books|          5|            0|          0|   N|                N|Great Twist on Zo...|I've known about ...| 2012-05-03|
|         US|   44244451| RZKRFS2UUMFFU|031088926X|     676347131|Sticky Faith Teen...|           Books|          5|    

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])

In [7]:
vine_total_votes = vine_df.filter(vine_df.total_votes >= 20)

In [18]:
vine_total_votes.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R18VIM840CEFRP|          1|           16|        105|   N|                N|
|R14PMAJTY0EAAT|          4|          135|        142|   N|                Y|
|R1363VA3TPNLVB|          5|          370|        388|   N|                Y|
| RBQZC5A3TSWT5|          5|           11|         22|   N|                Y|
| RW00TDPV9U93E|          1|           35|         76|   N|                N|
|R3OW0AIVLEDIQ7|          3|           99|        121|   N|                N|
|R3DTESO4FUAPKQ|          5|           90|        102|   N|                N|
|R18I0XBYWFSR5T|          1|           16|         50|   N|                N|
|R1VSQ3QK30VDI0|          5|           23|         35|   N|                N|
|R3NEJBF4FXMVUU|          5|           87|         90|   N|     

In [20]:
vine_help_votes = vine_total_votes.filter((vine_total_votes.helpful_votes) >= 50)

In [21]:
vine_help_votes.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R14PMAJTY0EAAT|          4|          135|        142|   N|                Y|
|R1363VA3TPNLVB|          5|          370|        388|   N|                Y|
|R3OW0AIVLEDIQ7|          3|           99|        121|   N|                N|
|R3DTESO4FUAPKQ|          5|           90|        102|   N|                N|
|R3NEJBF4FXMVUU|          5|           87|         90|   N|                Y|
|R2DLFW0GMIFTKP|          5|           53|         58|   N|                Y|
| RQZ638E0E34Z3|          2|          259|        299|   N|                N|
| RE4BOBQLRIJU2|          1|          163|        221|   N|                N|
|R1504NWBRNYN0B|          1|          949|       1029|   N|                Y|
|R2Z8083V8ZFQVZ|          3|           82|         87|   Y|     

In [22]:
vine_paid = vine_help_votes.filter(vine_help_votes.vine == "Y")

In [23]:
vine_paid.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2Z8083V8ZFQVZ|          3|           82|         87|   Y|                N|
|R2RY328TIDXMTE|          2|           50|         61|   Y|                N|
|R3FO8YQ2BKL1RG|          5|           93|         94|   Y|                N|
|R2ZD6GIDCKAJNM|          5|          118|        139|   Y|                N|
|R3DKQQNX6X3VTU|          4|           53|         56|   Y|                N|
| RHGNFBA3HATNJ|          5|          223|        237|   Y|                N|
| RK49ETJM70RY1|          5|          244|        259|   Y|                N|
| RGFM0JV76IWTR|          5|           85|         94|   Y|                N|
|R2MVTQRK9MA8HQ|          5|          246|        257|   Y|                N|
|R3W1SRJ7P448QG|          5|          665|        691|   Y|     

In [24]:
vine_not_paid = vine_help_votes.filter(vine_help_votes.vine == "N")

In [25]:
vine_not_paid.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R14PMAJTY0EAAT|          4|          135|        142|   N|                Y|
|R1363VA3TPNLVB|          5|          370|        388|   N|                Y|
|R3OW0AIVLEDIQ7|          3|           99|        121|   N|                N|
|R3DTESO4FUAPKQ|          5|           90|        102|   N|                N|
|R3NEJBF4FXMVUU|          5|           87|         90|   N|                Y|
|R2DLFW0GMIFTKP|          5|           53|         58|   N|                Y|
| RQZ638E0E34Z3|          2|          259|        299|   N|                N|
| RE4BOBQLRIJU2|          1|          163|        221|   N|                N|
|R1504NWBRNYN0B|          1|          949|       1029|   N|                Y|
| RL1R0S35GUGP2|          5|           62|         65|   N|     

In [26]:
vine_paid_5s = vine_paid.filter(vine_paid.star_rating == 5).count()

In [28]:
vine_paid_5s

568

In [29]:
vine_not_paid_5s = vine_not_paid.filter(vine_not_paid.star_rating == 5).count()

In [30]:
vine_not_paid_5s

35532

In [31]:
vine_paid_total = vine_paid.count()

In [32]:
vine_paid_total

1344

In [33]:
vine_not_paid_total = vine_not_paid.count()

In [34]:
vine_not_paid_total

69043

In [35]:
vine_paid_percentage = vine_paid_5s / vine_paid_total

In [37]:
vine_paid_percentage * 100

42.26190476190476

In [38]:
vine_not_paid_percentage = vine_not_paid_5s / vine_not_paid_total

In [39]:
vine_not_paid_percentage * 100

51.46358066712049